In [ ]:
def segment_logo_skimage(image):
    # Apply Otsu's thresholding
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    thresh = threshold_otsu(gray_image)
    bw = closing(gray_image > thresh, square(2))

    # Remove artifacts connected to image border
    cleared = clear_border(bw)

    # Label image regions
    label_image = label(cleared)

    # Create an overlay image with transparent background
    image_label_overlay = label2rgb(label_image, image=image, bg_label=0)

    # Find the logo region
    regions = regionprops(label_image)
    logo_region = max(regions, key=lambda r: r.area)  # Assuming the logo is the largest region

    # Create a mask for the logo
    logo_mask = np.zeros_like(gray_image)
    minr, minc, maxr, maxc = logo_region.bbox
    logo_mask[minr:maxr, minc:maxc] = label_image[minr:maxr, minc:maxc] == logo_region.label

    # Extract the logo from the original image
    logo = np.zeros_like(image)
    for i in range(3):  # for each color channel
        logo[:, :, i] = image[:, :, i] * logo_mask

    # Change background to a solid color (e.g., white)
    background_color = [255, 255, 255]  # White background
    for i in range(3):  # for each color channel
        logo[:, :, i][logo_mask == 0] = background_color[i]

    return logo

image = cv2.imread('./handbag3.jpeg')
logo = segment_logo_skimage(image)
# Display the original image and the extracted logo side by side
fig, ax = plt.subplots(1, 2, figsize=(12, 6))

# Original image
ax[0].imshow(image)
ax[0].set_title('Original Image')
ax[0].axis('off')

# Extracted logo
ax[1].imshow(logo)
ax[1].set_title('Extracted Logo')
ax[1].axis('off')

plt.show()